![image](Gen_AI_banner_no_text.png)


# Introduction

Hi 👋,
I'm David Kunz, an architect for the SAP Cloud Application Programming Model (CAP) and in this course, you will learn some of the prompt-engineering techniques I created in the development of [capGPT](https://github.tools.sap/cap/AI/tree/main/cds-gen), a CLI tool to generate apps for the [SAP Cloud Application Programming Model](https://cap.cloud.sap/docs/).
With these techniques, you will be able to incorporate large language models into your own applications in a more robust and efficient way.

## SAP Cloud Application Programming Model (CAP)

To get an understanding of the basic building blocks of CAP, please read the [Getting Started Guide](https://cap.cloud.sap/docs/get-started/in-a-nutshell).
It's not mandatory for this session, but it's helpful to know the overall context.

TLDR:
- [CAP](https://cap.cloud.sap/docs/) is a framework to build cloud-native apps on BTP.
- It comes with a declarative language to model your domain entities, example CDS file:

    ```cds
    entity Book {
        key ID: UUID;
        title: String;
        author: Association to Author;
    }

    entity Author {
        key ID: UUID;
        firstName: String;
        lastName: String
    }

    service Bookshop {
        entity Books as projection on Book;
        entity Authors as projection on Author;
    }
    ```
- You can provide test data in form of CSV files, example:

    ```csv
    ID;title;author_ID
    fc61e102-f0e2-47b8-9853-ed007dcd52e7;The Great Gatsby;4a3cf450-3512-4efd-afb9-2567e140d251
    f72871b6-6e80-40e4-a2c2-08dbad077bdf;To Kill a Mockingbird;64240dcc-b7a6-41d0-a925-4a7e2b842989
    383c8a33-f335-405e-8e05-b5710aa926c0;Pride and Prejudice;d10dd37b-e4c6-4141-b394-2962d32a3546
    ```

- With these building blocks you already have a working [OData](https://www.odata.org/) service.
- You can enrich your CDS model with UI annotations and add a [Fiori Elements](https://experience.sap.com/fiori-design-web/smart-templates/) UI on top.


## capGPT

On November 30th, 2022, [ChatGPT](https://chat.openai.com/) was released and a few days later I noticed that it could even generate CAP models, [see my corresponding blog post](https://blogs.sap.com/2022/12/08/having-fun-with-sap-cap-and-chatgpt/).
A few months later, on a Friday evening, I built the initial version of [capGPT](https://github.tools.sap/cap/AI/tree/main/cds-gen) which I refined in the following months. The plan is to ship it to external customers for TechEd 2023.

### Usage

You can generate CDS data models and test data just by providing a textual description, giving you a head start into CAP development.

```bash
capGPT "Build an app with authors and books. An author has a first name and a last name. A book has a title and an author."
```

This will automatically generate all needed CDS files and test data for a working server, including a Fiori Elements preview on top.

## Agenda

In the following sections, I will give you a step-by-step overview on how to design the prompts, overcome the obstacles and make it as efficient as possible.

Note: All code snippets are written in Python, but you can use any programming language since you effectively just need to send HTTP requests. `capGPT` itself is a Node.js application.

# Helper Function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
from llm_commons.btp_llm.openai import BTPChatCompletion

def get_completion(messages, model="gpt-35-turbo", temperature=0):
    response = BTPChatCompletion.create(
        deployment_id=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message["content"]

## Test It
Try it out and see if it works.

Note: For more complicated prompts, it might take a lot longer to compute.

In [ ]:
print(get_completion([{ "role": "user", "content":"What is 1 + 1?" }]))

You are encouraged to run all examples to get a feeling of the processing time and how the results vary. Play around with the prompts, see how it changes the results.

# Step 1: The Naïve Approach

Before spending time too much time in prompt engineering, it's better to find a good use case first. As a rule of thumb, you should find tasks which don't require too much internal knowledge, otherwise you'd need to utilize other techniques like Retrieval Augmented Generation (RAG) and/or fine-tuning.

CDS models mostly require a semantic understanding of the world, hence they're a good candidate.

Let's see how the most basic prompt performs.

In [ ]:
print(get_completion([{ "role": "user", "content":"Create a CAP CDS Model for authors and books." }]))

The result is disappointing. Let's try again with more details about CAP. It usually helps to begin with "You are an expert for ...".

In [ ]:
print(get_completion([{ "role": "user", "content":"""
  You are an expert for the SAP Cloud Application Programming Model,
  create a CDS Model for authors and books.
  """ }]))

It works! At least sometimes. There are still some quirks, e.g. it sometimes gets confused with ABAP CDS (another flavor of CDS) or it generates invalid syntax. But that's okay for now. It shows that the use case is valid, in principle.

Our host program needs to extract the code and ignore the surrounding text (which the LLM usually adds). So let's change the prompt to embed the code in a \`\`\`cds ... \`\`\` code fence.

In [ ]:
print(get_completion([{ "role": "user", "content":"""
You are an expert for the SAP Cloud Application Programming Model,
create a CDS Model for authors and books.
Use cds code fences in the following format:
```cds
...
```
Do not add any additional text.
""" }]))

Now, with the help of some regular expression (e.g. `/```cds\n([\s\S]*?)\n```/gi`), we get our code and we can write it to the local file system (which we're not going to do in this notebook).

We also need service projections, so we can ask it to also generate those.

In [ ]:
print(get_completion([{ "role": "user", "content":"""
You are an expert for the SAP Cloud Application Programming Model,
create a CDS Model for authors and books.
Also include service projections.
Use cds code fences in the following format:
```cds
...
```
Do not add any additional text.
""" }]))

# Excercise 1: Generate Two Code Blocks

In CAP, we usually split up the entity and service definitions into two files. Adjust the prompt to generate two separate code blocks.

In [ ]:
print(get_completion([{ "role": "user", "content":"""
You are an expert for the SAP Cloud Application Programming Model,
create a CDS Model for authors and books.
Also include service projections. 

###########################
<ADD YOUR INSTRUCTION HERE>
###########################

Use cds code fences in the following format:
```cds
...
```
Do not add any additional text.
""" }]))

# Step 2: Validating the Output

The previous prompt works quite well, but the resulting CDS models tend to have errors. We all know how pedantic compilers are.

It's not a good user experience if the output is wrong and the user is required to manually delete all files.

Let's increase the reboustness by adding a validation and retry mechanism.

In [ ]:
def check_result(result):
    # Here, we could use the CDS compiler to validate the model, let's just return True for now.
    return True

def retry_get_completion(messages, max_retries=5):
    for _ in range(max_retries):
        result = get_completion(messages)
        if check_result(result):
            return result
        print("Result is wrong, retrying...")
    print("Max retries reached, giving up.")
    return None

Depending on the scenario, it might help to feed the validation error back into the conversation. Then the LLM knows about it and can improve the result.

# Step 3: Guidance

We can improve the results by giving more guidance, e.g. by providing rules to overcome common mistakes or by showing some examples (few-shot prompting).



In [ ]:
print(retry_get_completion([{ "role": "user", "content":"""
You are an expert for the SAP Cloud Application Programming Model,
create a CDS Model for authors and books.
Also include service projections. 

Example:
```cds
entity Order {
    key ID: UUID;
    date: DateTime;
    customer: Association to Customer;
    items: Composition of many OrderItem on items.order = $self;
}
entity OrderItem {
    key ID: UUID;
    order: Association to Order;
    product: String(200);
}
entity Customer {
    key ID: UUID;
    name: String(200);
}
service OrderService {
    entity Orders as projection on Order;
    entity Customers as projection on Customer;
}
```
Rules:
- associations/compositions are defined in the entity
- possible types are: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)

Use cds code fences in the following format:
```cds
...
```
Do not add any additional text.
""" }]))

Since we don't know what the LLM already knows, providing more guidelines is an iterative process. First we need to identify the common mistakes through example generations and only then we can derive a list of rules to overcome them.

# Step 3: Intermediate Representation

Depending on the scenario, the list of rules can quickly grow, consuming more and more (input) tokens. The main problem with this approach is that the degrees of freedom are quite high, we accept _any_ CAP CDS model (in compliance with the rules/prompt).
To make it more robust and reliable, we need to _reduce_ the degrees of freedom.
With the current approach, it's also hard to repair the output. We would need to parse it into an abstract syntax tree (if possible) and validate/repair it and since we don't know _what_ CDS features it incorporates, writing such a program is tedious.

We also rely on the LLM's knowledge of CAP. The whole approach would fail if it wouldn't know about it.

A possible solution to these problems is to introduce an _intermediate representation_. A data structure, which we define, where we're in full control of what properties need to be generated, which we can optimize and fine-tune for LLM usage.
The format is not really important, but [JSON](https://www.json.org/json-en.html) is a good choice since the token count is quite low (more on that later) and the training data of LLMs usually include quite a lot of JSON objects, so the format is well known.


In [ ]:
print(retry_get_completion([{ "role": "user", "content":"""
You are a domain modeling expert, create a domain model for authors and books. 

Example:
```json
{
  "namespace": "inventory",
  "entities": [
    {
      "name": "Order",
      "elements": [
        {
          "name": "ID",
          "type": "UUID"
        },
        {
          "name": "date",
          "type": "DateTime"
        }
      ],
      "associations": [
        {
          "name": "customer",
          "target": "Customer",
          "multiplicity": "one",
          "selfContained": false
        },
        {
          "name": "items",
          "target": "OrderItem",
          "multiplicity": "many",
          "backlink": "order",
          "selfContained": true
        }
      ]
    },
    {
      "name": "OrderItem",
      "elements": [
        {
          "name": "ID",
          "type": "UUID"
        },
        {
          "name": "product",
          "type": "String(200)"
        }
      ],
      "associations": [
        {
          "name": "order",
          "target": "Order",
          "multiplicity": "one",
          "selfContained": false
        }
      ]
    },
    {
      "name": "Customer",
      "elements": [
        {
          "name": "ID",
          "type": "UUID"
        },
        {
          "name": "name",
          "type": "String(200)"
        }
      ]
    }
  ]
}
```
Rules:
- possible types are: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)

Use json code fences in the following format:
```json
...
```
Do not add any additional text.
""" }]))

Notice that CAP isn't even mentioned in the system prompt. The LLM only needs to know how to construct this JSON object with meaningful content. No CAP knowledge required.

We can now write tools to validate the JSON object and repair it. For example, we could remove all elements with invalid types. Or we could ensure that all backlinks exist.
Usually, many problems can be solved programmatically. This is a common guideline: Only use LLMs where they're good at. Rely on conventional programming for the rest.

After the output is checked and repaired, we can create the CDS files programmatically.


# Excercise 2: Add More Features

Once the base structure yields robust results, we can add more features successively. In this excercise, modify the structure to add labels to all elements, that means human-readable text to identify the field.

In [ ]:
print(retry_get_completion([{ "role": "user", "content":"""
You are a domain modeling expert, create a domain model for authors and books. 

###########################
<Modify the following JSON to include labels>
###########################
Example:
```json
{
  "namespace": "inventory",
  "entities": [
    {
      "name": "Order",
      "elements": [
        {
          "name": "ID",
          "type": "UUID"
        },
        {
          "name": "date",
          "type": "DateTime"
        }
      ],
      "associations": [
        {
          "name": "customer",
          "target": "Customer",
          "multiplicity": "one",
          "selfContained": false
        },
        {
          "name": "items",
          "target": "OrderItem",
          "multiplicity": "many",
          "backlink": "order",
          "selfContained": true
        }
      ]
    },
    {
      "name": "OrderItem",
      "elements": [
        {
          "name": "ID",
          "type": "UUID"
        },
        {
          "name": "product",
          "type": "String(200)"
        }
      ],
      "associations": [
        {
          "name": "order",
          "target": "Order",
          "multiplicity": "one",
          "selfContained": false
        }
      ]
    },
    {
      "name": "Customer",
      "elements": [
        {
          "name": "ID",
          "type": "UUID"
        },
        {
          "name": "name",
          "type": "String(200)"
        }
      ]
    }
  ]
}
```
Rules:
- possible types are: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)

Use json code fences in the following format:
```json
...
```
Do not add any additional text.
""" }]))

# Step 4: JSON Schema

As an alternative to providing examples, we can also use [JSON schema](https://json-schema.org/) notation to describe the data format. This is also regularly used in [OpenAI's Function Calling Approach](https://openai.com/blog/function-calling-and-other-api-updates).
It doesn't strictly need to be JSON schema, you can also invent new properties if needed.
One of the advantages is that you can add property descriptions, making it easier for LLMs to generate the respective values.

In [ ]:
print(retry_get_completion([{ "role": "user", "content":"""
You are a domain modeling expert, create a domain model for authors and books following the JSON schema. 

```json_schema
{
  "description": "the domain model",
  "type": "object",
  "properties": {
    "namespace": {
      "type": "string"
    },
    "entities": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string"
          },
          "elements": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string"
                },
                "type": {
                  "description": "possible values: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)",
                  "type": "string"
                }
              },
              "required": ["name", "type"]
            }
          },
          "associations": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string"
                },
                "target": {
                  "type": "string"
                },
                "multiplicity": {
                  "type": "string"
                },
                "selfContained": {
                  "type": "boolean"
                },
                "backlink": {
                  "description": "must point to existing association of target",
                  "type": "string"
                }
              },
              "required": ["name", "target", "multiplicity", "selfContained"]
            }
          }
        },
        "required": ["name", "elements", "associations"]
      }
    }
  },
  "required": ["namespace", "entities"]
}
```
Based on this schema, generate the JSON object, do not output any text, just the JSON, format:

```json
{{generated JSON}}
```
""" }]))

# Exercise 3: Fix Identifiers

Modify the JSON schema to only allow alphanumeric values for entity and element names.

In [ ]:
print(retry_get_completion([{ "role": "user", "content":"""
You are a domain modeling expert, create a domain model for authors and books following the JSON schema. 

###########################
<Modify the following JSON schema to only allow alphanumeric values for entity and element names>
###########################

```json_schema
{
  "description": "the domain model",
  "type": "object",
  "properties": {
    "namespace": {
      "type": "string"
    },
    "entities": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string"
          },
          "elements": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string"
                },
                "type": {
                  "description": "possible values: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)",
                  "type": "string"
                }
              },
              "required": ["name", "type"]
            }
          },
          "associations": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string"
                },
                "target": {
                  "type": "string"
                },
                "multiplicity": {
                  "type": "string"
                },
                "selfContained": {
                  "type": "boolean"
                },
                "backlink": {
                  "description": "must point to existing association of target",
                  "type": "string"
                }
              },
              "required": ["name", "target", "multiplicity", "selfContained"]
            }
          }
        },
        "required": ["name", "elements", "associations"]
      }
    }
  },
  "required": ["namespace", "entities"]
}
```
Based on this schema, generate the JSON object, do not output any text, just the JSON, format:

```json
{{generated JSON}}
```
""" }]))

# Step 5: Minimizing Token Count

Compared to the original approach where we generate CDS models directly, it's becoming more and more verbose and we consume a lot input tokens. Let's see how we can improve this.
One of the advantages of the JSON format is that we can minimize it. This drastically reduces the token count as can be seen in [OpenAI's Tokenizer tool](https://platform.openai.com/tokenizer):

![img](./tokens.png)

You can see that each whitespace character is equivalent to one token. By minimizing the JSON object, we can reduce the amount of tokens, in this example it's reduced by about 80%.
Therefore, all input and output JSON objects should be minimized. For output tokens, this is especially important because the processing time is linear with the number of output tokens.
The less we generate, the faster it is. Let's apply it.

Notice that the JSON format doesn't add much overhead to the token count.

In [ ]:
print(retry_get_completion([{ "role": "user", "content":"""
You are a domain modeling expert, create a domain model for authors and books following the JSON schema. 

```json_schema
{"description":"the domain model","type":"object","properties":{"namespace":{"type":"string"},"entities":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"elements":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"type":{"description":"possible values: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)","type":"string"}},"required":["name","type"]}},"associations":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"target":{"type":"string"},"multiplicity":{"type":"string"},"selfContained":{"type":"boolean"},"backlink":{"description":"must point to existing association of target","type":"string"}},"required":["name","target","multiplicity","selfContained"]}}},"required":["name","elements","associations"]}}},"required":["namespace","entities"]}
```
Based on this schema, generate the JSON object, do not output any text, just the JSON, format:

```json
{{generated JSON}}
```

It must not be formatted, everything in one line, minimize the token count
""" }]))

An alternative is using [OpenAI's Function Calling method](https://openai.com/blog/function-calling-and-other-api-updates) by providing a function with one argument, described by a JSON schema. Unfortunately, OpenAI doesn't minify the resulting function call, therefore I cannot recommend this approach.

# Step 6: Separating User Input

Now we need to separate the user input from the system message. Please keep in mind that user prompts can always 'override' the system message, hence malicious users might introduce unwanted behaviour (prompt injection), but that's of no concern in this use case.

In [ ]:
print(retry_get_completion([{ "role": "system", "content":"""
You are a domain modeling expert, create a domain model for the user's prompt, following the JSON schema. 

```json_schema
{"description":"the domain model","type":"object","properties":{"namespace":{"type":"string"},"entities":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"elements":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"type":{"description":"possible values: UUID,Boolean,Integer,Decimal(precision, scale),Double,Date,Time,DateTime,Timestamp,String(length)","type":"string"}},"required":["name","type"]}},"associations":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"target":{"type":"string"},"multiplicity":{"type":"string"},"selfContained":{"type":"boolean"},"backlink":{"description":"must point to existing association of target","type":"string"}},"required":["name","target","multiplicity","selfContained"]}}},"required":["name","elements","associations"]}}},"required":["namespace","entities"]}
```
Based on this schema, generate the JSON object, do not output any text, just the JSON, format:

```json
{{generated JSON}}
```

It must not be formatted, everything in one line, minimize the token count
""" }, { "role": "user", "content": "create a bookshop model with authors and books" }]))

# Step 7: Generate Test Data

In order to generate test data, let's start with a simple prompt where we only provide the CDS model.

In [ ]:
print(retry_get_completion([{ "role": "system", "content":"""
You are an expert for the SAP Cloud Application Programming Model.

Generate test data for the following CDS model.
```cds
namespace my.bookstore;

entity Author {
    key ID: UUID;
    name: String(200);
    books: Composition of many Book on books.author = $self;
}

entity Book {
    key ID: UUID;
    title: String(200);
    author: Association to Author;
}

service BookstoreService {
    entity Authors as projection on Author;
    entity Books as projection on Book;
}
```

The format must be csv files.
""" }]))

The result is close and with proper rules we could generate correct test data, but the approach is not robust enough.
Instead of providing the CDS model, we could also use tools to make it as easy as possible for the machine to generate data.

One option is to use the CDS compiler to generate SQL table definitions, a format well understood by GPT. If we use the HANA
flavour, we even get the corresponding foreign-key relations. Let's try it.

```bash
cds compile '*' -2 hdbtable
```
```sql
----- my.bookstore.Author.hdbtable -----
COLUMN TABLE my_bookstore_Author (
  ID NVARCHAR(36) NOT NULL,
  name NVARCHAR(200),
  PRIMARY KEY(ID)
) WITH ASSOCIATIONS (
  MANY TO MANY JOIN my_bookstore_Book AS books ON (books.author_ID = ID)
)
----- my.bookstore.Book.hdbtable -----
COLUMN TABLE my_bookstore_Book (
  ID NVARCHAR(36) NOT NULL,
  title NVARCHAR(200),
  author_ID NVARCHAR(36),
  PRIMARY KEY(ID)
) WITH ASSOCIATIONS (
  MANY TO ONE JOIN my_bookstore_Author AS author ON (author.ID = author_ID)
)
```

With the tool `cds add data`, we can also create CSV files which already contain the correct header line:

```bash
cds add data

Adding feature 'data'...
Creating db/data/my.bookstore-Author.csv
Creating db/data/my.bookstore-Book.csv

Successfully added features to your project.
```


Let's include the results of those tools into the system prompt.

In [ ]:
print(retry_get_completion([{ "role": "system", "content":"""
You are a data analyst. Based on the given database tables create content for the empty CSV files,
generate at least 5 rows.

Tables:
```sql
----- my.bookstore.Author.hdbtable -----
COLUMN TABLE my_bookstore_Author (
  ID NVARCHAR(36) NOT NULL,
  name NVARCHAR(200),
  PRIMARY KEY(ID)
) WITH ASSOCIATIONS (
  MANY TO MANY JOIN my_bookstore_Book AS books ON (books.author_ID = ID)
)
----- my.bookstore.Book.hdbtable -----
COLUMN TABLE my_bookstore_Book (
  ID NVARCHAR(36) NOT NULL,
  title NVARCHAR(200),
  author_ID NVARCHAR(36),
  PRIMARY KEY(ID)
) WITH ASSOCIATIONS (
  MANY TO ONE JOIN my_bookstore_Author AS author ON (author.ID = author_ID)
)

Empty CSV files:
```json
{"db/data/my.bookstore-Author.csv": "ID,name","db/data/my.bookstore-Book.csv": "ID,title,author_ID"}
```

The result must be in JSON format in the same structure as the empty csv files.
Add the rows after the header line, separated by \n characters.

```json
<result>
```

It must not be formatted, everything in one line, minimize the token count
""" }]))

This looks better.

# Step 8: Fixing UUID Values

Unfortunately, the UUID values are not correct. They have to be in UUID v4 format, e.g. `5c14d42e-25fe-4539-8678-59a447ec9bf0`. It turns out that GPT regularly fails to generate valid UUID v4 values, even when specifying the exact format in the prompt.
One approach which always leads to proper UUID values is to let the LLM generate _placeholder values_ which are then replaced programmatically by the host program.

In [ ]:
import re
import uuid

def replace_uuid_placeholders(input_string):
    uuid_mapping = {}  # Dictionary to store generated UUIDs for each placeholder
    
    # Find all occurrences of @UUID followed by a number
    uuid_placeholders = re.findall(r'@UUID\d+', input_string)
    
    # Generate and store UUIDs for each placeholder if not already generated
    for placeholder in uuid_placeholders:
        if placeholder not in uuid_mapping:
            uuid_mapping[placeholder] = str(uuid.uuid4())
    
    # Replace each placeholder with its corresponding UUID
    for placeholder, uuid_value in uuid_mapping.items():
        input_string = input_string.replace(placeholder, uuid_value)
    
    return input_string


print(replace_uuid_placeholders(retry_get_completion([{ "role": "system", "content":"""
You are a data analyst. Based on the given database tables create content for the empty CSV files.

Tables:
```sql
----- my.bookstore.Author.hdbtable -----
COLUMN TABLE my_bookstore_Author (
  ID NVARCHAR(36) NOT NULL,
  name NVARCHAR(200),
  PRIMARY KEY(ID)
) WITH ASSOCIATIONS (
  MANY TO MANY JOIN my_bookstore_Book AS books ON (books.author_ID = ID)
)
----- my.bookstore.Book.hdbtable -----
COLUMN TABLE my_bookstore_Book (
  ID NVARCHAR(36) NOT NULL,
  title NVARCHAR(200),
  author_ID NVARCHAR(36),
  PRIMARY KEY(ID)
) WITH ASSOCIATIONS (
  MANY TO ONE JOIN my_bookstore_Author AS author ON (author.ID = author_ID)
)

Empty CSV files:
```json
{"db/data/my.bookstore-Author.csv": "ID,name","db/data/my.bookstore-Book.csv": "ID,title,author_ID"}
```

For all NVARCHAR(36) columns, generate placeholder values @UUID1, @UUID2, etc.

It must not be formatted, everything in one line, minimize the token count
""" }])))

# Summary

In this session you've learned how to query large language models, guide them to produce the desired output, reduce the amount of tokens in the generation, utilize tools to reduce complexity and introduce intermediate representations for more control.

# Congratulations

Congratulations on completing the hands-on activity! 🎉 We're really proud of your progress. 

Please take a moment to fill out this **[anonymous quick poll](https://sapit-home-prod-004.launchpad.cfapps.eu10.hana.ondemand.com/site#Quick-Poll&/item/f33819c3-81ea-4153-90f4-11bc7c09621c)**. 
Remember, your feedback remains completely anonymous and will only be used to enhance the learning experience.

Thank you for your time and keep up the good work! 🙌 
Your learning partner, SAP Dev. Learning Team ailearning@sap.com